Compute the Probability of Each census group having rodent issues. 

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from pgmpy.models import BayesianModel
from pgmpy.estimators import BayesianEstimator
from pgmpy.estimators import HillClimbSearch
from pgmpy.models import BayesianNetwork

In [2]:
# loading the merged data 
df = pd.read_csv('..\\Data\\rodents_per_year_merged.csv')

In [3]:
# show the dummy variables
df.head()

,spatial_id,year,l_Commercial_sum,l_Other_sum,l_Outdoor_sum,l_Residential_sum,l_Residential-Mixed_sum,l_Vacant_Space_sum,d_Friday_sum,d_Monday_sum,...,c_Caribbean_sum,c_Chinese_sum,c_Coffee/Tea_sum,c_Italian_sum,c_Japanese_sum,c_Latin American_sum,c_Mexican_sum,c_Pizza_sum,c_other_sum,num_violations
0,360050001000,2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,360050001000,2021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,360050001000,2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,360050001000,2018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,360050001000,2019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
len(df)

38958

In [5]:
df.columns

Index(['spatial_id', 'year', 'l_Commercial_sum', 'l_Other_sum',
       'l_Outdoor_sum', 'l_Residential_sum', 'l_Residential-Mixed_sum',
       'l_Vacant_Space_sum', 'd_Friday_sum', 'd_Monday_sum', 'd_Saturday_sum',
       'd_Sunday_sum', 'd_Thursday_sum', 'd_Tuesday_sum', 'd_Wednesday_sum',
       't_Evening_sum', 't_Midday_sum', 't_Morning_sum', 'num_sightings',
       's_Dead_Animal:Residential_sum', 's_Dead_Animal:Street_sum',
       's_Dog_waste:Street_sum', 's_Illegal_Dumping:Street_sum',
       's_Trash:Residential_sum', 's_Trash:Street_sum',
       's_Trash_MissedService:Street_sum', 's_Trash_Overflowing:Street_sum',
       's_Trash_Time:Street_sum', 's_Trash_Unsecure:Residential_sum',
       's_Trash_Unsecure:Street_sum', 'num_dsny_complaints', 'subway_count',
       'v_flies_sum', 'v_rats/mice_sum', 'v_roaches_sum', 'c_American_sum',
       'c_Bakery Products/Desserts_sum', 'c_Caribbean_sum', 'c_Chinese_sum',
       'c_Coffee/Tea_sum', 'c_Italian_sum', 'c_Japanese_sum',
      

In [6]:
# train test split
train, test = train_test_split(df, test_size=0.2) # 20% test data
train.head()

,spatial_id,year,l_Commercial_sum,l_Other_sum,l_Outdoor_sum,l_Residential_sum,l_Residential-Mixed_sum,l_Vacant_Space_sum,d_Friday_sum,d_Monday_sum,...,c_Caribbean_sum,c_Chinese_sum,c_Coffee/Tea_sum,c_Italian_sum,c_Japanese_sum,c_Latin American_sum,c_Mexican_sum,c_Pizza_sum,c_other_sum,num_violations
37666,360850128062,2019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28647,360810212001,2018,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36498,360811463001,2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4957,360050358007,2021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33353,360810731001,2023,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# train a bayesian net 
# find the best structure
hc = HillClimbSearch(train)
best_model = hc.estimate(scoring_method='BicScore')
print(best_model.edges())

  0%|          | 0/1000000 [00:00<?, ?it/s]

[('year', 's_Trash_Unsecure:Residential_sum'), ('year', 's_Trash_Time:Street_sum'), ('year', 'v_flies_sum'), ('year', 's_Dead_Animal:Street_sum'), ('year', 's_Illegal_Dumping:Street_sum'), ('year', 't_Morning_sum'), ('year', 'num_dsny_complaints'), ('year', 'c_Chinese_sum'), ('year', 'c_Bakery Products/Desserts_sum'), ('l_Residential_sum', 'num_sightings'), ('d_Monday_sum', 'l_Other_sum'), ('d_Tuesday_sum', 'l_Residential-Mixed_sum'), ('num_sightings', 't_Evening_sum'), ('num_sightings', 't_Midday_sum'), ('num_sightings', 'd_Monday_sum'), ('num_sightings', 'd_Tuesday_sum'), ('num_sightings', 'd_Sunday_sum'), ('num_sightings', 'd_Thursday_sum'), ('num_sightings', 'd_Wednesday_sum'), ('num_sightings', 'd_Friday_sum'), ('num_sightings', 'd_Saturday_sum'), ('s_Trash_MissedService:Street_sum', 'year'), ('v_flies_sum', 'num_violations'), ('v_rats/mice_sum', 'c_Pizza_sum'), ('v_rats/mice_sum', 'c_American_sum'), ('v_rats/mice_sum', 'c_Coffee/Tea_sum'), ('v_roaches_sum', 'c_Mexican_sum'), ('nu

In [8]:
# visualize the best structure with graphviz

# train the model with the best structure
best_model = BayesianNetwork(hc.estimate(scoring_method='BicScore').edges())
best_model.fit(train, estimator=BayesianEstimator, prior_type="K2")

  0%|          | 0/1000000 [00:00<?, ?it/s]

In [9]:
for cpd in best_model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)
for thenode in best_model.nodes():
    print(best_model.local_independencies(thenode))

CPD of year:
+----------------------------------+-----+
| s_Trash_MissedService:Street_sum | ... |
+----------------------------------+-----+
| year(2018)                       | ... |
+----------------------------------+-----+
| year(2019)                       | ... |
+----------------------------------+-----+
| year(2020)                       | ... |
+----------------------------------+-----+
| year(2021)                       | ... |
+----------------------------------+-----+
| year(2022)                       | ... |
+----------------------------------+-----+
| year(2023)                       | ... |
+----------------------------------+-----+
CPD of s_Trash_Unsecure:Residential_sum:
+-----+------------------------+
| ... | year(2023)             |
+-----+------------------------+
| ... | 0.9965497412305923     |
+-----+------------------------+
| ... | 0.00019168104274487253 |
+-----+------------------------+
| ... | 0.00019168104274487253 |
+-----+------------------------+
| ..

In [11]:
# predict spatial id with the best structure
train_x = train.loc[:, "year": "num_violations"]
predicted_train = best_model.predict(train_x)
predicted_train

ValueError: Data has variables which are not in the model

In [14]:
# print the nodes of the model 
best_model.nodes()

NodeView(('year', 's_Trash_Unsecure:Residential_sum', 's_Trash_Time:Street_sum', 'v_flies_sum', 's_Dead_Animal:Street_sum', 's_Illegal_Dumping:Street_sum', 't_Morning_sum', 'num_dsny_complaints', 'c_Chinese_sum', 'c_Bakery Products/Desserts_sum', 'l_Residential_sum', 'num_sightings', 'd_Monday_sum', 'l_Other_sum', 'd_Tuesday_sum', 'l_Residential-Mixed_sum', 't_Evening_sum', 't_Midday_sum', 'd_Sunday_sum', 'd_Thursday_sum', 'd_Wednesday_sum', 'd_Friday_sum', 'd_Saturday_sum', 's_Trash_MissedService:Street_sum', 'num_violations', 'v_rats/mice_sum', 'c_Pizza_sum', 'c_American_sum', 'c_Coffee/Tea_sum', 'v_roaches_sum', 'c_Mexican_sum', 'c_other_sum'))

Using just Bayesian Network might not be ideal for calculating the probability for each of the census block group. We should use other additional methods for calculations. 

In [16]:
# checking values in num_violations 
df.num_violations.value_counts()

# creating a new columns labelling if rodent problems are present or not in the census block group
df['rodent_problem'] = df['num_violations'].apply(lambda x: 1 if x > 0 else 0)
df.head() 

,spatial_id,year,l_Commercial_sum,l_Other_sum,l_Outdoor_sum,l_Residential_sum,l_Residential-Mixed_sum,l_Vacant_Space_sum,d_Friday_sum,d_Monday_sum,...,c_Chinese_sum,c_Coffee/Tea_sum,c_Italian_sum,c_Japanese_sum,c_Latin American_sum,c_Mexican_sum,c_Pizza_sum,c_other_sum,num_violations,rodent_problem
0,360050001000,2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,360050001000,2021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,360050001000,2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,360050001000,2018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,360050001000,2019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
